In [ ]:
import os, sys, copy, time, math, random, numbers, itertools, tqdm, importlib, re
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import rasterio
import torch
import yaml

from sklearn import metrics
from skimage import transform as trans
from pathlib import Path
from collections.abc import Sequence
from datetime import datetime, timedelta
from scipy.ndimage import rotate
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch import optim
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# The code cell is used to add the src directory to the Python path, making 
# it possible to import modules from that directory. 

module_path = os.path.abspath(os.path.join('../src'))
sys.path.insert(0, module_path)

In [ ]:
from custom_dataset import CropData
from models.unet import Unet
from model_compiler import ModelCompiler
from utils import *

In [ ]:
# The code cell loads a configuration file (default_config.yaml) using the YAML library and stores the 
# configuration data in the config dictionary. Then, it processes the global_stats section of the config 
# dictionary by expanding the lists for each stats based on the number of available time points. 
# As you can see we decided to generate a single set of normalization statistics and use it to 
# normalize all the time-points.

yaml_config_path = "/home/workdir/config/default_config.yaml"  # replace this path to your own config file.
num_time_points = 3  # Change this number accordingly if you use a dataset with a different temporal length.

with open(yaml_config_path, 'r') as file:
    config = yaml.load(file, Loader=yaml.SafeLoader)

# Perform multiplication and concatenation for each key in global_stats
for key, value in config['global_stats'].items():
    config['global_stats'][key] = value * num_time_points

In [ ]:
# OPTIONAL
# pretty-print the config dictionary

import pprint
pprint.pprint(config, width=100, compact=True)

In [ ]:
config['criterion']

### Steps for training and finetuning

In [ ]:
# STEP 1-1: Data Preparation and Loading
# This code cell reads the input dataset for training, pre-processes it, and creates a 'torch.utils.Dataset' 
# object to be used in the model training process. It utilizes the 'CropData' class from the custom module 
# (CropData.py) to achieve this.

train_dataset = CropData(src_dir=config["src_dir"],
                         usage="train",
                         dataset_name=config["train_dataset_name"],
                         csv_path=config["train_csv_path"],
                         apply_normalization=config["apply_normalization"],
                         normal_strategy=config["normal_strategy"],
                         stat_procedure=config["stat_procedure"],
                         global_stats=config["global_stats"],
                         trans=config["transformations"], 
                         **config["aug_params"])

In [ ]:
# OPTIONAL

show_random_patches(train_dataset, sample_num=3, rgb_bands=(3, 2, 1))

In [ ]:
# OPTIONAL
# Check the distribution of categories in a dataset

labels_count = get_labels_distribution(train_dataset, num_classes=14, ignore_class=0)
plot_labels_distribution(labels_count)

In [ ]:
# STEP 1-2: Data Batching and Device Assignment
# This code cell batchifies the training dataset using the 'DataLoader' class from PyTorch. The 'DataLoader' 
# efficiently loads the 'train_dataset' in batches, facilitating memory management during training.

train_loader = DataLoader(train_dataset,
                          batch_size=config["train_BatchSize"], 
                          shuffle=True)

In [ ]:
# STEP 1-3
# Read the input dataset for validation, pre-process it and generate a 'torch.utils.Dataset' object 
# that can be accepted to get loaded into the model.

val_dataset = CropData(src_dir=config["src_dir"],
                       usage="validation",
                       dataset_name=config["train_dataset_name"],
                       csv_path=config["val_csv_path"],
                       apply_normalization=config["apply_normalization"],
                       normal_strategy=config["normal_strategy"],
                       stat_procedure=config["stat_procedure"],
                       global_stats=config["global_stats"],)

In [ ]:
# OPTIONAL
# Check the distribution of categories in a dataset

labels_count = get_labels_distribution(val_dataset)
plot_labels_distribution(labels_count)

In [ ]:
# STEP 1-4
# Batchify the validation dataset and put it on the defined 'Device'.

val_loader = DataLoader(val_dataset, 
                        batch_size=config["val_test_BatchSize"], 
                        shuffle=False)

In [ ]:
# STEP 2: Model Initialization
# The model is based on the U-Net architecture, a popular choice for image segmentation tasks. 

model = Unet(n_classes=config["n_classes"], 
             in_channels=config["input_channels"], 
             use_skipAtt=config["use_skipAtt"],
             filter_config=config["filter_config"],
             dropout_rate=config["train_dropout_rate"])

In [ ]:
# STEP 3: Model Compilation and Training Setup
# This code cell compiles the deep learning model, making it ready for training and evaluation. The 'ModelCompiler' 
# class is employed for this purpose, which offers a set of essential functionalities, such as model training 
# ('fit'), model saving, accuracy evaluation ('accuracy_evaluation'), inference ('inference'), and checkpointing 
# with resume capabilities. Additionally, it generates tensorboard reports and graphs for monitoring the training 
# process.

# Note: For training from scratch pass the "params_init" and "freeze_params" as None which results in loading a vanilla
# model with random initialization based on the "model_init_type" parameter. For training with warmup, only change the
# "params_init" to the path to model's saved parameter (not a checkpoint). For fine-tuning, change the "params_init" to 
# the path to model's saved parameter (by default its in the chckpt folder in working directory) and pass a list of integers
# representing the index of layers to be frozen.

compiled_model = ModelCompiler(model,
                               working_dir=config["working_dir"],
                               out_dir=config["out_dir"],
                               num_classes=config["n_classes"],
                               inch=config["input_channels"],
                               class_mapping=config["class_mapping"],
                               gpu_devices=config["gpuDevices"],
                               model_init_type=config["init_type"], 
                               params_init=config["params_init"],
                               freeze_params=config["freeze_params"])

In [ ]:
# STEP 4: Model Training and Validation
# This code cell trains and validates the compiled deep learning model for the specified number of epochs. 
# The model is trained using the "fit" method provided by the 'compiled_model' object. The training data 
# is obtained from 'train_loader', which contains batched samples and labels. Additionally, the 'val_loader' 
# is used for model validation during training. For each epoch the model reports the average loss value for both 
# training and validation phases, providing insights into its generalization capabilities.

# During the training the "fit" method will create a folder called 'chkpt' inside your working directory
# and save the checkpoints in user-defined intervals in that folder.
# Note: In case the training process is interrupted, you can change the "resume" argument and pass in the
# epoch to resume based on the created checkpoints.

criterion_name = config['criterion']['name']
weight = config['criterion']['weight']
ignore_index = config['criterion']['ignore_index']
gamma = config['criterion']['gamma']

if criterion_name == 'TverskyFocalLoss':
    criterion = TverskyFocalLoss(weight=weight, ignore_index=ignore_index, gamma=gamma)
else:
    criterion = TverskyFocalLoss(weight=weight, ignore_index=ignore_index)
    

compiled_model.fit(train_loader,
                   val_loader, 
                   epochs=config["epochs"], 
                   optimizer_name=config["optimizer"], 
                   lr_init=config["LR"],
                   lr_policy=config["LR_policy"], 
                   criterion=criterion, 
                   momentum=config["momentum"],
                   checkpoint_interval=config["checkpoint_interval"],
                   resume=config["resume"],
                   resume_epoch=config["resume_epoch"],
                   **config["lr_prams"])

In [ ]:
# STEP 5
# Same model parameters
compiled_model.save(save_object="params")

In [ ]:
# STEP 6
# Generate the accuracy metrics.
metrics = compiled_model.accuracy_evaluation(val_loader, filename=config["val_metric_fname"])

### Steps for prediction

In [ ]:
test_dataset = CropData(src_dir=config["src_dir"],
                       usage="inference",
                       dataset_name=config["train_dataset_name"],
                       csv_path=config["test_csv_path"],
                       apply_normalization=config["apply_normalization"],
                       normal_strategy=config["normal_strategy"],
                       stat_procedure=config["stat_procedure"],
                       global_stats=config["global_stats"],)

In [ ]:
def meta_handling_collate_fn(batch):
    images = []
    labels = []
    img_ids = []
    img_metas = []

    # Unpack elements from each sample in the batch
    for sample in batch:
        images.append(sample[0])
        labels.append(sample[1])
        img_ids.append(sample[2])
        img_metas.append(sample[3])  # append the dict to the list

    # Stack images and labels into a single tensor
    images = torch.stack(images, dim=0)
    labels = torch.stack(labels, dim=0)
    
    return images, labels, img_ids, img_metas


test_loader = DataLoader(test_dataset, 
                        batch_size=config["val_test_BatchSize"], 
                        shuffle=False,
                        collate_fn=meta_handling_collate_fn)

In [ ]:
images, labels, img_ids, img_metas = next(iter(test_loader))

In [ ]:
img_metas[0]

In [ ]:
img_metas["transform"] = [t.item() for t in img_metas["transform"]]

In [ ]:
t = img_metas["transform"]
len(t)

In [ ]:
model = Unet(n_classes=config["n_classes"], 
             in_channels=config["input_channels"], 
             use_skipAtt=config["use_skipAtt"],
             filter_config=config["filter_config"],
             dropout_rate=config["train_dropout_rate"])

In [ ]:
compiled_model = ModelCompiler(model,
                               working_dir=config["working_dir"],
                               out_dir=config["out_dir"],
                               num_classes=config["n_classes"],
                               inch=config["input_channels"],
                               class_mapping=config["class_mapping"],
                               gpu_devices=config["gpuDevices"],
                               model_init_type=config["init_type"], 
                               params_init=config["params_init"],
                               freeze_params=config["freeze_params"])

In [ ]:
compiled_model.inference(test_loader, out_dir=config["out_dir"])

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure
import matplotlib.colors as mcolors

def plot_image_and_label(image_path, label_path, band_composite=(4,3,2), 
                         stretch=None, class_mapping=None):
    """
    Plots a loaded image and its corresponding label using matplotlib and 
    rasterio.plot.

    Args:
        image_path (str): Path to the image file.
        label_path (str): Path to the label file.
        stretch (str): A string specifying the contrast stretch to apply to the 
            image (e.g. 'linear', 'sqrt', or 'log').
        band_composite (tuple of int): A tuple specifying the band indices to 
            use for an RGB image (e.g. (3, 2, 1)).
        class_mapping (dict): A dictionary mapping class values to class names.

    Returns:
        None
    """
    # Open the image and label files
    with rasterio.open(image_path) as src:
        image_array = src.read()

    with rasterio.open(label_path) as src:
        label_array = src.read(1)

    # Set up the figure and axes
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

    # Transpose the image_array if it's 3D and the first dimension is the smallest
    if (image_array.ndim == 3) and (image_array.shape[0] == np.min(image_array.shape)):
        image_array = image_array.transpose(1,2,0)
    
    image_size = [image_array.shape[0], image_array.shape[1]]
    label_size = [label_array.shape[0], label_array.shape[1]]

    # If a band composite is specified, create an RGB image
    red_band = image_array[:, :, band_composite[0] - 1] / np.max(image_array[:, :, band_composite[0] - 1])
    green_band = image_array[:, :, band_composite[1] - 1] / np.max(image_array[:, :, band_composite[1] - 1])
    blue_band = image_array[:, :, band_composite[2] - 1] / np.max(image_array[:, :, band_composite[2] - 1])

    # Stack the bands to create an RGB image
    rgb_image = np.stack([red_band, green_band, blue_band], axis=-1)

    # Apply the contrast stretch if specified
    if stretch:
        p_min, p_max = np.percentile(rgb_image, (2, 98))
        stretched = exposure.rescale_intensity(
            rgb_image, in_range=(p_min, p_max), out_range=(0, 1)
        )
    else:
        stretched = rgb_image

    # Display the RGB image using matplotlib
    axs[0].imshow(stretched)
    
    # Add a title and axis labels
    axs[0].set_title('Image')
    axs[0].set_xlabel(f'# Column: {image_size[0]}')
    axs[0].set_ylabel(f'# Row: {image_size[1]}')
    axs[0].axis('off')
    axs[0].set_aspect('equal')

    # If class mapping is provided, create a colormap and a colorbar for the classes
    if class_mapping is not None:
        # Create a color map for your classes
        cmap = mcolors.ListedColormap(['black', 'mediumseagreen', 'lime', 'forestgreen', 'yellow', 
                                        'yellowgreen', 'cyan', 'crimson', 'darkblue', 'wheat', 
                                        'goldenrod', 'gold', 'lightsteelblue', 'sandybrown', 
                                        'lightgray', 'darkorange', 'red'])

    # Plot label chip
    im = axs[1].imshow(label_array, cmap=cmap)
    axs[1].set_title('Label')
    axs[1].set_xlabel(f'# Column: {label_size[0]}')
    axs[1].set_ylabel(f'# Row: {label_size[1]}')

    # Create colorbar after imshow
    if class_mapping is not None:
        colorbar = fig.colorbar(im, ax=axs[1], ticks=list(class_mapping.keys()), label='class')
        colorbar.ax.set_yticklabels(list(class_mapping.values()))  # set the colorbar labels to be the class names

    # Show the plot
    axs[1].axis('off')
    axs[1].set_aspect('equal')
    plt.show()


In [ ]:
image_path = "/home/data/chips_filtered/chip_140_494_merged.tif"
label_path = "/home/workdir/output2/hardened_prob/crisp_id_140_494.tif"
plot_image_and_label(image_path, label_path, band_composite=(4,3,2), stretch="linear", class_mapping=config["class_mapping"])